In [2]:
import chess
import chess.pgn
pgn = open("data/Carlsen.pgn")

In [13]:
first_game = chess.pgn.read_game(pgn)

In [14]:
first_game.headers

Headers(Event='Troll Masters', Site='Gausdal NOR', Date='2001.01.06', Round='2', White='Carlsen,Magnus', Black='Brameld,A', Result='0-1', WhiteElo='', BlackElo='2095', ECO='B05')

In [ ]:
games = []
game = []
for move in first_game.mainline_moves():
    game.append(move.uci())
games.append(game)
games

In [20]:
games[0][0]

'e2e4'

In [27]:
file_map = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
rank_map = {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7}
move = list(games[0][0])
move = file_map[move[0]]*1000+rank_map[move[1]]*100+file_map[move[2]]*10+rank_map[move[3]]*1

In [36]:
move

4143

In [40]:
move // 1000 % 10, move // 100 % 10, move // 10 % 10, move % 10

(4, 1, 4, 3)

In [37]:
def get_pos_nums(num):
    pos_nums = []
    while num != 0:
        pos_nums.append(num % 10)
        num = num // 10
    return pos_nums

get_pos_nums(move)

[3, 4, 1, 4]

In [98]:
import numpy as np

class Encoding():
    def __init__(self) -> None:
        self.n_vocab = 4098 # 4096 + 2
        self.itof = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}
        self.itor = {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8'}
        self.vocab = {0: '<s>', '<s>': 0, 1: '</s>', '</s>': 1}
        index = 2
        for i in range(8):
            for j in range(8):
                for k in range(8):
                    for l in range(8):
                        uci = self.itof[i] + self.itor[j] + self.itof[k] + self.itor[l]
                        self.vocab[index] = uci
                        self.vocab[uci] = index
                        index += 1
        for i in range(8):
            for piece in ["b", "n", "r", "q"]:
                if i != 0:
                    uci = f"{self.itof[i]}7{self.itof[i-1]}8{piece}"
                    self.vocab[index] = uci
                    self.vocab[uci] = index
                    index += 1
                if i != 7:
                    uci = f"{self.itof[i]}7{self.itof[i+1]}8{piece}"
                    self.vocab[index] = uci
                    self.vocab[uci] = index
                    index += 1
                uci = f"{self.itof[i]}7{self.itof[i]}8{piece}"
                self.vocab[index] = uci
                self.vocab[uci] = index
                index += 1

                if i != 0:
                    uci = f"{self.itof[i]}2{self.itof[i-1]}1{piece}"
                    self.vocab[index] = uci
                    self.vocab[uci] = index
                    index += 1
                if i != 7:
                    uci = f"{self.itof[i]}2{self.itof[i+1]}1{piece}"
                    self.vocab[index] = uci
                    self.vocab[uci] = index
                    index += 1
                uci = f"{self.itof[i]}2{self.itof[i]}2{piece}"
                self.vocab[index] = uci
                self.vocab[uci] = index
                index += 1
        print(index)
    
    def encode(self, move: str): return self.vocab[move]
    def decode(self, move: int): return self.vocab[move]


In [123]:
itof = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}
itor = {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8'}
vocab = {0: '<s>', '<s>': 0, 1: '</s>', '</s>': 1}
index = 2
for i in range(8):
    for j in range(8):
        for k in range(8):
            for l in range(8):
                uci = itof[i] + itor[j] + itof[k] + itor[l]
                vocab[index] = uci
                vocab[uci] = index
                index += 1

for i in range(8):
    for piece in ['b', 'n', 'q', 'r']:
        if i != 0:
            uci = itof[i] + '2' + itof[i-1] + '1' + piece
            vocab[index] = uci
            vocab[uci] = index
            index += 1
        if i != 7:
            uci = itof[i] + '2' + itof[i+1] + '1' + piece
            vocab[index] = uci
            vocab[uci] = index
            index += 1
        uci = itof[i] + '2' + itof[i] + '1' + piece
        vocab[index] = uci
        vocab[uci] = index
        index += 1

for i in range(8):
    for piece in ['b', 'n', 'q', 'r']:
        if i != 0:
            uci = itof[i] + '7' + itof[i-1] + '8' + piece
            vocab[index] = uci
            vocab[uci] = index
            index += 1
        if i != 7:
            uci = itof[i] + '7' + itof[i+1] + '8' + piece
            vocab[index] = uci
            vocab[uci] = index
            index += 1
        uci = itof[i] + '7' + itof[i] + '8' + piece
        vocab[index] = uci
        vocab[uci] = index
        index += 1
import pickle
with open("vocab.pickle", "wb") as f:
    pickle.dump(vocab, f)

In [127]:
len(vocab.items()) // 2

4274

In [128]:
import pickle

vocab = None
with open('vocab.pickle', "rb") as f:
    vocab = pickle.load(f)
n_vocab = len(vocab.items()) // 2

class Encoding():
    def __init__(self) -> None:
        self.n_vocab = n_vocab
    def encode(self, move: str): return vocab[move]
    def decode(self, move: int): return vocab[move]


In [129]:
#enc = Encoding()
tokens = []
x = []
with open('data/Carlsen.pgn') as pgn:
    while game := chess.pgn.read_game(pgn):
        #tokens.append(enc.encode("<s>"))
        for move in game.mainline_moves():
            x.append(move.uci())
            tokens.append(vocab[move.uci()])
        #tokens.append(enc.encode("</s>"))
len(tokens)
#n_vocab = enc.n_vocab

523622

In [130]:
x == [vocab[tok] for tok in tokens]

True

In [85]:
enc.encode('a1a0')

2

In [93]:
for m in x:
    if len(m) > 4 and list(m)[-1] != "q":
        print(m)

d7d8r
a2a1b
e7d8r
e7e8b
e2f1n
d2d1n
h2h1n
c7c8n
e7e8r
b2b1b
d2c1r
g7f8r
c2c1n
c7c8n
f2f1r
g2g1r


In [137]:
import os
[f for f in os.listdir('data') if os.path.isfile(os.path.join('data', f)) and os.path.splitext(f)[1] == ".pgn"]

['Carlsen.pgn']